# Motion Detection

In this file, movement detection is firstly applied to pick the motion CSI slice. Then a 1DCNN is trained and used to classify after necessary signal processing.

## get walk CSI data

In [ ]:
def CSIextracter(folder,motion):
    #1. load CSI data and SVM model
#     movement_folder = '/Users/wanrylin/Master Programme/dataset/original data/movement'
#     folder = movement_folder
#     motion = 'walk'
    movement_listpath = os.listdir(folder)
    movement_listpath = [x for x in movement_listpath if motion in x]
    
    CSI_amp = []
    CSI_pha = []
    Time = []
    label = []
    # Load the csi sequence
    for moving_num in range(len(movement_listpath)):
        filename = folder + '/' + movement_listpath[moving_num]
        CSIdata = pd.read_csv(filename)
        CSI_array = CSIdata.values
        time_sequence = CSI_array[1:,0]
        csi_data_str = CSI_array[1:,1:]
        # change str into complex
        csi_data = np.zeros_like(csi_data_str, dtype=np.complex64)
        [row,col] = csi_data_str.shape
        for i in range(row):
            for j in range(col):
                element = csi_data_str[i,j]
                element = element.replace('i', 'j')
                csi_data[i,j] = complex(element)

        # revise time sequence
        peak = [ time_sequence[t] for t in range(1,len(time_sequence)-1) if time_sequence[t] > time_sequence[t-1] and time_sequence[t] > time_sequence[t+1]]
        peak = []
        for i in range(1,len(time_sequence)-1):
            if time_sequence[i] > time_sequence[i-1] and time_sequence[i] > time_sequence[i+1]:
                peak.append(i)
        for position in peak:
            time_sequence[int(position+1):] += 4096 
        time_sequence -= (time_sequence[0]-1)
        # check time sequence is it in order
        flag = 0
        for i in range(1,len(time_sequence)-1):
            if time_sequence[i] < time_sequence[i-1]: 
                flag = 1
                print("the %d in time sequence not in order",i)
                break
        Time.append(time_sequence)

        # get amplitude data only
        csi_amp = np.abs(csi_data)
        CSI_amp.append(csi_amp)

        # revise CSI phase data
        # phase correction from [IndoTrack]
        rx1 = csi_data[:,:57]
        rx2 = csi_data[:,57:]
        csi_data_adj = np.zeros(rx1.shape, dtype=complex)
        csi_data_ref_adj = np.zeros(rx2.shape, dtype=complex)
        alpha_sum = 0
        for jj in range(57):
            amp = np.abs(rx1[:,jj])
            alpha = np.min(amp[amp!=0])
            alpha_sum = alpha_sum + alpha
            csi_data_adj[:,jj] = np.abs(np.abs(rx1[:,jj]) - alpha) * np.exp(1j * np.angle(rx1[:,jj]))
        beta = 5000 * alpha_sum / 57 # 5000 is an experience value, it just need to be large enough
        for jj in range(57):
            csi_data_ref_adj[:,jj] = (np.abs(rx2[:,jj]) + beta) * np.exp(1j * np.angle(rx2[:,jj]))
        # Conjunction Multiply
        conj_mult = csi_data_adj * np.conj(csi_data_ref_adj)
        csi_revised = np.zeros(conj_mult.shape, dtype=complex)
        for j in range(57):
            for i in range(len(conj_mult)):
                z = conj_mult[i,j]
                if z.imag < 0:
                    csi_revised[i,j] = -1 * z.real + (-1j) * z.imag
                else:
                    csi_revised[i,j] = z           
        csi_phase = np.angle(csi_revised)
        CSI_pha.append(csi_phase)
        
        # generate the label for the motion
        file_name = os.path.basename(filename)
        name_list = file_name.split('_')
        if len(name_list) > 1:
            name_list = name_list[:-1]
            motion_label = '_'.join(name_list)
        else:
            motion_label = name_list[0]
        label.append(motion_label)
        
    return CSI_amp,CSI_pha,Time,label

## movement detection

In [ ]:
def movement_detection(CSI_pha,Time):
    
    svm_folder = '/Users/wanrylin/Master Programme/dataset/movement detection/svm_model'
    # load svm model
    filename = svm_folder + '/pca_svm_model.joblib'
    svm = joblib.load(filename)
    filename = svm_folder + '/pca_scaler.joblib'
    scaler = joblib.load(filename)
    #3. movement period detection
    feature_num = 2
    dimension_num = 10
    # cut the csi sequence into frame with 1 second long and 0.5 interval
    sampling_rate = 200 # Hz
    frame_length = 1 # second
    frame = frame_length * sampling_rate
    interval_length = 0.5 # second
    interval = interval_length * sampling_rate
    
    # detection
    Label = []
    for k in range(len(Time)):
        csi_phase = CSI_pha[k]
        time_sequence = Time[k]
        time_sequence = list(time_sequence)
        num = int((time_sequence[-1] - frame)/interval + 1) # number of frames
        start = 0 # initial boundry
        end = 150
        labels = np.zeros((len(time_sequence),))
        # prediction for each frame
        for i in range(num):
            #1. generate frame
            if (i * interval + 1) in time_sequence:
                start = int(time_sequence.index(i * interval + 1))
            else:
                start_time = np.min([x for x in time_sequence[start:start + 200] if x > (i * interval + 1)])
                start = int(time_sequence.index(start_time))
            if (i * interval + frame + 1) in time_sequence or (i * interval + frame + 1) > time_sequence[-1]:
                end = int(time_sequence.index(np.min([i * interval + frame + 1,time_sequence[-1]])))
            else:
                end_time = np.min([x for x in time_sequence[end:end + 200] if x > (i * interval + frame + 1)])
                end = int(np.min([time_sequence.index(end_time),len(time_sequence)]))
            phase = csi_phase[start:end]

            #2. Make predictions on the test set using the loaded model
            #(a) create csi feature
            # create a PCA object with 10 components
            pca = PCA(dimension_num)
            # fit the PCA object to the data and transform the data
            phase_pca = pca.fit_transform(phase)
            # iterate each component
            feature_matrix = np.zeros((feature_num, dimension_num))
            for j in range(dimension_num):
                pha_mean = np.mean(phase_pca[:, j])
                pha_var = np.var(phase_pca[:, j])
                sub_feature = np.array([pha_mean, pha_var])
                feature_matrix[:, j] = sub_feature
            feature_matrix = feature_matrix.T
            feature = feature_matrix.reshape(1,-1)   
            #(b) scale the feature
            feature_scaled = scaler.transform(feature)
            #(c) predict label
            pred_label = svm.predict(feature_scaled)
            #(d) store the label in labels
            if pred_label == 1:
                for n in range(start,end):
                    labels[n] = 1

        Label.append(labels)
        
    return Label

## get the moving slice

In [ ]:
import pandas as pd
import numpy as np
import os

movement_folder = '/Users/wanrylin/Master Programme/dataset/original data/movement'
motion = 'walk'
CSI_amp,CSI_pha,Time_sequence,motion_label = CSIextracter(movement_folder,motion)

In [ ]:
import joblib
from sklearn.decomposition import PCA

Label = movement_detection(CSI_pha,Time_sequence)

## generate dataset

In [ ]:
# butter worth low pass filter
def butter_lowpass_filter(data, cutoff, freq, order=5):
    nyq = 0.5 * freq
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False, fs = freq)
    
    [row,col] = data.shape
    filtered_data = np.zeros(data.shape)
    for i in range(col):
        wave = data[:,i]
        filtered = lfilter(b, a, wave)
        filtered_data[:,i] = filtered
    return filtered_data

In [ ]:
def dataset_generater(CSI_amp,Label):
    # from sklearn.preprocessing import StandardScaler
    # from scipy.signal import resample
    new_N = 200
    dimension_num = 10
    num = len(CSI_amp)
    dataset = np.zeros((num,new_N,dimension_num))
    for i in range(num):
        csi_amp = CSI_amp[i]
        labels = Label[i].astype(np.int32)
        labels = list(labels)
        
        # cut the motion CSI slice
        motion_start = labels.index(1)
        motion_end = len(labels) - labels[::-1].index(1) - 1  
        csi_slice = csi_amp[motion_start:motion_end]
        
        # PCA 
        # create a PCA object with 10 components
        pca = PCA(dimension_num)
        # fit the PCA object to the data and transform the data
        amp_pca = pca.fit_transform(csi_slice)
        
        # standarize
        # Create an instance of StandardScaler
        scaler = StandardScaler()
        scaler.fit(amp_pca)
        # Transform the data using the scaler
        scaled_data = scaler.transform(amp_pca)
        
        # downsample
        downsampled_amp = resample(scaled_data, new_N, axis=0)
        
        # low pass filter
        order = 6
        ori_sampling_rate = 200 #Hz
        fs = len(amp_pca)/new_N * ori_sampling_rate # sample rate, Hz
        cutoff = 100 # desired cutoff frequency of the filter, Hz # Get the filter coefficients so we can check its frequency response.
        filtered_amp = butter_lowpass_filter(downsampled_amp, cutoff, fs, order)
        
        # store
        dataset[i] = filtered_amp
        
    return dataset

In [ ]:
def dataset_booster(CSI_amp,Label,motion_label):
    # from sklearn.preprocessing import StandardScaler
    # from scipy.signal import resample
    boost = 11
    new_N = 200
    dimension_num = 10
    num = len(CSI_amp)
    dataset = np.zeros((num * boost,new_N,dimension_num))
    data_label = np.zeros(num * boost)
    boost_back = [x - 5 if x > 5 else 0 for x in range(boost)]
    boost_front = boost_back[::-1]
    for i in range(num):
        csi_amp = CSI_amp[i]
        labels = Label[i].astype(np.int32)
        labels = list(labels)
        
        for j in range(boost):
            # cut the motion CSI slice
            motion_start = labels.index(1)
            motion_end = len(labels) - labels[::-1].index(1) - 1
            motion_start = np.max([0,motion_start - 10*boost_front[j]])
            motion_end = np.min([len(labels),motion_end + 10*boost_back[j]])
            csi_slice = csi_amp[motion_start:motion_end]
    #         csi_slice = csi_amp

            # PCA 
            # create a PCA object with 10 components
            pca = PCA(dimension_num)
            # fit the PCA object to the data and transform the data
            amp_pca = pca.fit_transform(csi_slice)

            # standarize
            # Create an instance of StandardScaler
            scaler = StandardScaler()
            scaler.fit(amp_pca)
            # Transform the data using the scaler
            scaled_data = scaler.transform(amp_pca)

            # downsample
            downsampled_amp = resample(scaled_data, new_N, axis=0)

            # low pass filter
            order = 6
            ori_sampling_rate = 200 #Hz
            fs = len(amp_pca)/new_N * ori_sampling_rate # sample rate, Hz
            cutoff = 100 # desired cutoff frequency of the filter, Hz # Get the filter coefficients so we can check its frequency response.
            filtered_amp = butter_lowpass_filter(downsampled_amp, cutoff, fs, order)

            # store
            dataset[i*boost +j] = filtered_amp 
            data_label[i*boost +j] = motion_label[i]
        
    return dataset,data_label

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from scipy.signal import resample
import tensorflow as tf
from scipy.signal import butter, lfilter, freqz

# dataset = dataset_generater(CSI_amp,Label)
# motion = ['walk_approach','walk_leave']
# data_label = [motion.index(x) for x in motion_label]

# # Split data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(dataset, data_label, test_size=0.3, random_state=21)

# y_train = tf.keras.utils.to_categorical(y_train, 2)
# y_test = tf.keras.utils.to_categorical(y_test, 2)

Motion = ['walk_approach','walk_leave']
data_label = [Motion.index(x) for x in motion_label]
train_amp = CSI_amp[1::5] + CSI_amp[2::5] + CSI_amp[3::5] + CSI_amp[4::5]
train_label = Label[1::5] + Label[2::5] + Label[3::5] + Label[4::5]
train_motion_label = data_label[1::5] + data_label[2::5] + data_label[3::5] + data_label[4::5]
X_train,y_train = dataset_booster(train_amp,train_label,train_motion_label)

test_amp = CSI_amp[::5]
test_label = Label[::5]
test_motion_label = data_label[::5]
# X_test,y_test = dataset_booster(test_amp,test_label,test_motion_label)
X_test = dataset_generater(test_amp,test_label)
y_test = test_motion_label


# Split data into training and testing sets
# dataset = dataset_generater(CSI_amp,my_label)
# X_train, X_test, y_train, y_test = train_test_split(dataset, data_label, test_size=0.2, random_state=40)

y_train = tf.keras.utils.to_categorical(y_train, 2)
y_test = tf.keras.utils.to_categorical(y_test, 2)

In [ ]:
X_train.shape

In [ ]:
X_test.shape

## build 1DCNN

In [ ]:
from tensorflow.keras import layers
# Define input shape
input_shape = (200,10)  # N can vary, so set it to None

# Build model
model = tf.keras.Sequential([
    layers.Input(shape=input_shape),
    layers.Conv1D(32, 3, activation='relu', padding='same'),
    layers.MaxPooling1D(2),
    layers.Conv1D(16, 3, activation='relu', padding='same'),
    layers.MaxPooling1D(2),
    layers.Dropout(0.5),
    layers.Flatten(),
    layers.Dense(2, activation='softmax')
])

# Compile model
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# Print model summary
model.summary()

## train the NN

In [ ]:
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import ModelCheckpoint

# create a ModelCheckpoint callback to save the weights
model_folder = '/Users/wanrylin/Master Programme/python code/model/walk_1DCNN'
checkpoint = ModelCheckpoint(model_folder + "/model_weights.{epoch:02d}.h5",
                             save_weights_only=True)

# Create a TensorBoard callback and specify the log directory:
log_dir = "logs/fit/"
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)
# Pass the callback to the fit method of the model:
model_history = model.fit(X_train, y_train, epochs=50, batch_size=4, validation_split=0.2, callbacks=[checkpoint])


In [ ]:
# evaluate the model on the test dataset
test_loss, test_accuracy = model.evaluate(X_test,y_test,verbose=2)

In [ ]:
# get predicted probabilities for each class
y_pred_prob = model.predict(X_test)

# get predicted labels (class with highest probability)
y_pred = np.argmax(y_pred_prob, axis=1)

y_truth = np.argmax(y_test, axis=1)
print(y_pred)
print(y_truth)

In [ ]:
# Save the entire model as a SavedModel.
!mkdir -p saved_model
model.save('saved_model/walk_NN')

In [ ]:
csi_phase = CSI_pha[1]
time_sequence = Time_sequence[1]
# show 18th subcarrier 
fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1)
ax1.scatter(time_sequence, csi_phase[:,18],s = 1)
ax1.set_title("CSI phase")
ax2.scatter(time_sequence, csi_amp[:,18],s = 1)
ax2.set_title("CSI amplitude")
fig.subplots_adjust(hspace=0.5)
plt.show()

In [ ]:
motion_start

In [ ]:
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

csi_amp = CSI_amp[i]
labels = list(Label[i])
motion = motion_label[i]
time_sequence = Time_sequence[i]

# cut the motion CSI slice
motion_start = labels.index(1)
motion_end = len(labels) - labels[::-1].index(1) - 1
csi_slice = csi_amp[motion_start:motion_end]
time_sequence = time_sequence[motion_start:motion_end]
colors = list(np.zeros((len(csi_amp))))
for n in range(len(labels)):
    x = labels[n]
    if x == 0:
        colors[n] = 'b'
    if x == 1:
        colors[n] = 'r'

dimension_num = 10
# create a PCA object with 10 components
pca = PCA(dimension_num)
# fit the PCA object to the data and transform the data
amp_pca = pca.fit_transform(csi_slice)

# standarize
# Create an instance of StandardScaler
scaler = StandardScaler()
scaler.fit(amp_pca)
# Transform the data using the scaler
scaled_data = scaler.transform(amp_pca)

# show nth subcarrier 
print(i,motion)
fig, (ax1, ax2, ax3, ax4, ax5) = plt.subplots(nrows=5, ncols=1,figsize = (6,10))
ax1.scatter(time_sequence, scaled_data[:,0],s = 1)
ax1.set_title("PCA component 1")
ax2.scatter(time_sequence, scaled_data[:,1],s = 1)
ax2.set_title("PCA component 2")
ax3.scatter(time_sequence, scaled_data[:,2],s = 1)
ax3.set_title("PCA component 3")
ax4.scatter(time_sequence, scaled_data[:,3],s = 1)
ax4.set_title("PCA component 4")
ax5.scatter(time_sequence, scaled_data[:,4],s = 1)
ax5.set_title("PCA component 5")
fig.subplots_adjust(hspace=0.5)
plt.show()

In [ ]:
from scipy.signal import resample

new_N = 200
downsampled_amp = resample(scaled_data, new_N, axis=0)

In [ ]:
new_sequence = range(new_N)
fig, (ax1, ax2, ax3, ax4, ax5) = plt.subplots(nrows=5, ncols=1,figsize = (6,10))
ax1.scatter(new_sequence, downsampled_amp[:,0],s = 1)
ax1.set_title("PCA component 1")
ax2.scatter(new_sequence, downsampled_amp[:,1],s = 1)
ax2.set_title("PCA component 2")
ax3.scatter(new_sequence, downsampled_amp[:,2],s = 1)
ax3.set_title("PCA component 3")
ax4.scatter(new_sequence, downsampled_amp[:,3],s = 1)
ax4.set_title("PCA component 4")
ax5.scatter(new_sequence, downsampled_amp[:,4],s = 1)
ax5.set_title("PCA component 5")
fig.subplots_adjust(hspace=0.5)
plt.show()

In [ ]:
from scipy.signal import butter, lfilter, freqz

def butter_lowpass_filter(data, cutoff, freq, order=5):
    nyq = 0.5 * freq
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False, fs = freq)
    
    [row,col] = data.shape
    filtered_data = np.zeros(data.shape)
    for i in range(col):
        wave = data[:,i]
        filtered = lfilter(b, a, wave)
        filtered_data[:,i] = filtered
    return filtered_data  # Filter requirements.


order = 6
fs = new_N/len(amp_pca) * 200 # sample rate, Hz
cutoff = 20 # desired cutoff frequency of the filter, Hz # Get the filter coefficients so we can check its frequency response.
filtered_amp = butter_lowpass_filter(downsampled_amp, cutoff, fs, order)
fig, (ax1, ax2, ax3, ax4, ax5) = plt.subplots(nrows=5, ncols=1,figsize = (6,10))
ax1.plot(new_sequence, filtered_amp[:,0])
ax1.set_title("PCA component 1")
ax2.plot(new_sequence, filtered_amp[:,1])
ax2.set_title("PCA component 2")
ax3.plot(new_sequence, filtered_amp[:,2])
ax3.set_title("PCA component 3")
ax4.plot(new_sequence, filtered_amp[:,3])
ax4.set_title("PCA component 4")
ax5.plot(new_sequence, filtered_amp[:,4])
ax5.set_title("PCA component 5")
fig.subplots_adjust(hspace=0.5)
plt.show()


In [ ]:
movement_folder = '/Users/wanrylin/Master Programme/dataset/original data/movement'
folder = movement_folder
motion = 'walk'
movement_listpath = os.listdir(folder)
movement_listpath = [x for x in movement_listpath if motion in x]
print(movement_listpath[1])

In [ ]:
import matplotlib.pyplot as plt
svm_folder = '/Users/wanrylin/Master Programme/dataset/movement detection/svm_model'
# load svm model
filename = svm_folder + '/pca_svm_model.joblib'
svm = joblib.load(filename)
filename = svm_folder + '/pca_scaler.joblib'
scaler = joblib.load(filename)
#3. movement period detection
feature_num = 2
dimension_num = 10
# cut the csi sequence into frame with 1 second long and 0.5 interval
sampling_rate = 200 # Hz
frame_length = 1 # second
frame = frame_length * sampling_rate
interval_length = 0.5 # second
interval = interval_length * sampling_rate

# detection
Label = []
k = 1
csi_phase = CSI_pha[k]
time_sequence = Time_sequence[k]
time_sequence = list(time_sequence)
num = int((time_sequence[-1] - frame)/interval + 1) # number of frames
start = 0 # initial boundry
end = 150
labels = np.zeros((len(time_sequence),))
colors = list(np.zeros((len(csi_phase))))
# prediction for each frame
for i in range(num):
    #1. generate frame
    if (i * interval + 1) in time_sequence:
        start = int(time_sequence.index(i * interval + 1))
    else:
        start_time = np.min([x for x in time_sequence[start:start + 200] if x > (i * interval + 1)])
        start = int(time_sequence.index(start_time))
    if (i * interval + frame + 1) in time_sequence or (i * interval + frame + 1) > time_sequence[-1]:
        end = int(time_sequence.index(np.min([i * interval + frame + 1,time_sequence[-1]])))
    else:
        end_time = np.min([x for x in time_sequence[end:end + 200] if x > (i * interval + frame + 1)])
        end = int(np.min([time_sequence.index(end_time),len(time_sequence)]))
    phase = csi_phase[start:end]
    

    #2. Make predictions on the test set using the loaded model
    #(a) create csi feature
    # create a PCA object with 10 components
    pca = PCA(dimension_num)
    # fit the PCA object to the data and transform the data
    phase_pca = pca.fit_transform(phase)
    # iterate each component
    feature_matrix = np.zeros((feature_num, dimension_num))
    for j in range(dimension_num):
        pha_mean = np.mean(phase_pca[:, j])
        pha_var = np.var(phase_pca[:, j])
        sub_feature = np.array([pha_mean, pha_var])
        feature_matrix[:, j] = sub_feature
    feature_matrix = feature_matrix.T
    feature = feature_matrix.reshape(1,-1)   
    #(b) scale the feature
    feature_scaled = scaler.transform(feature)
    #(c) predict label
    pred_label = svm.predict(feature_scaled)
    print(feature_scaled)
    #(d) store the label in labels
    if pred_label == 1:
        for n in range(start,end):
            labels[n] = 1
    if pred_label == 0:
        for n in range(start,end):
            colors[n] = 'b'
    elif pred_label == 1:
        for n in range(start,end):
            colors[n] = 'r'
    for n in range(end,len(colors)):
        colors[n] = 'black'

# show 18th subcarrier 
csi_amp = CSI_amp[k]
fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1)
ax1.scatter(time_sequence, csi_phase[:,48],s = 1, c = colors)
ax1.set_title("CSI phase")
ax2.scatter(time_sequence, csi_amp[:,48],s = 1,c = colors)
ax2.set_title("CSI amplitude")
fig.subplots_adjust(hspace=0.5)
plt.show()

In [ ]:
import pandas as pd
import numpy as np
import os

movement_folder = '/Users/wanrylin/Master Programme/dataset/original data/test/walking'
Motion = 'walk'
CSI_amp,CSI_pha,Time_sequence,motion_label = CSIextracter(movement_folder,Motion)

import joblib
from sklearn.decomposition import PCA

Label = movement_detection(CSI_pha,Time_sequence)

In [ ]:
# Load the saved model
model = tf.keras.models.load_model('saved_model/walk_NN')

In [ ]:
X_test= dataset_generater(CSI_AMP,my_test_label)
y_test = true_label
y_test = tf.keras.utils.to_categorical(y_test, 2)
test_loss, test_accuracy = model.evaluate(X_test,y_test,verbose=2)

In [ ]:
# get predicted probabilities for each class
y_pred_prob = model.predict(X_test)

# get predicted labels (class with highest probability)
y_pred = np.argmax(y_pred_prob, axis=1)

y_truth = np.argmax(y_test, axis=1)
print(y_pred)
print(y_truth)
for i in range(len(y_pred)):
    if y_pred[i] != y_truth[i]:
        print(Motion[y_pred[i]],Motion[y_truth[i]])

In [ ]:
i = 0
# my_test_label = []
# CSI_AMP = []
# true_label = []

In [ ]:
import matplotlib.pyplot as plt

csi_amp = CSI_amp[i]
csi_phase = CSI_pha[i]
labels = list(Label[i])
motion = motion_label[i]
time_sequence = Time_sequence[i]

# cut the motion CSI slice
motion_start = labels.index(1)
motion_end = len(labels) - labels[::-1].index(1) - 1
# motion_start = 0
# motion_end = 500
#     csi_slice = csi_amp[motion_start:motion_end]
#     time_sequence = time_sequence[motion_start:motion_end]
colors = list(np.zeros((len(csi_amp))))
for n in range(len(labels)):
    x = labels[n]
    if x == 0:
        colors[n] = 'b'
    if x == 1:
        colors[n] = 'r'

dimension_num = 10
# create a PCA object with 10 components
pca = PCA(dimension_num)
# fit the PCA object to the data and transform the data
amp_pca = pca.fit_transform(csi_amp)

# standarize
# Create an instance of StandardScaler
scaler = StandardScaler()
scaler.fit(amp_pca)
# Transform the data using the scaler
scaled_data = scaler.transform(amp_pca)


# show 18th subcarrier 
print(motion)
fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1)
ax1.scatter(time_sequence, csi_phase[:,18],s = 1, c = colors)
ax1.set_title("CSI phase")
ax2.scatter(time_sequence, scaled_data[:,0],s = 1,c = colors)
ax2.set_title("CSI amplitude")
fig.subplots_adjust(hspace=0.5)
plt.show()
print(motion_start,motion_end)

In [ ]:
i += 1

In [ ]:
time_sequence = list(time_sequence)
motion_start = time_sequence.index(3400)
motion_end = time_sequence.index(4200)
labels = np.zeros((len(time_sequence)))
for x in range(len(time_sequence)):
    if motion_start < x and x < motion_end:
        labels[x] = 1
my_test_label.append(labels)
CSI_AMP.append(csi_amp)
true_label.append(int(1))